In [1]:
import torch
import torchvision
import numpy as np
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(name)-12s %(levelname)-8s %(message)s")

from benchmark import ImageBenchmark
bench = ImageBenchmark()
models = list(bench.list_models())
models_dict = {}
for i, model in enumerate(models):
    if not model.torch_model_exists():
        continue
    print(f'{i}\t {model.__str__()}')
    models_dict[model.__str__()] = model

0	 pretrain(mbnetv2,ImageNet)-
1	 pretrain(resnet18,ImageNet)-
2	 train(mbnetv2,Flower102)-
3	 train(mbnetv2,SDog120)-
4	 train(resnet18,Flower102)-
5	 train(resnet18,SDog120)-
6	 pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.1)-
7	 pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.5)-
8	 pretrain(mbnetv2,ImageNet)-transfer(Flower102,1)-
9	 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.1)-
10	 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.5)-
11	 pretrain(mbnetv2,ImageNet)-transfer(SDog120,1)-
12	 pretrain(resnet18,ImageNet)-transfer(Flower102,0.1)-
13	 pretrain(resnet18,ImageNet)-transfer(Flower102,0.5)-
14	 pretrain(resnet18,ImageNet)-transfer(Flower102,1)-
15	 pretrain(resnet18,ImageNet)-transfer(SDog120,0.1)-
16	 pretrain(resnet18,ImageNet)-transfer(SDog120,0.5)-
17	 pretrain(resnet18,ImageNet)-transfer(SDog120,1)-
18	 pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.1)-quantize(qint8)-
19	 pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.1)-quantize(float16)-
20	 pretrain(mbnetv2,Im

In [83]:
from modeldiff import ModelDiff

def compare_with_seed(model1, model2, truth=-1):
    print(f'comparing:\n  model1={model1}\n  model2={model2}')
    md = ModelDiff(model1, model2)
    seed_inputs = md.get_seed_inputs(rand=False)
    sim = md.compute_similarity_with_inputs(seed_inputs)
    if truth == -1:
        truth = 1 if model1.__str__().split('-')[0] == model2.__str__().split('-')[0] else 0
    print(f' similarity is {sim}, truth is {truth}')

def test(compare):
    # compare(models[23], models[38], 1) # should be similar
    compare(models[1], models[0], 0)   # should be different
#     compare(models[13], models[25], 0) # should be different
#     compare(models[13], models[22], 0) # should be different
    compare(models[1], models[17], 1)  # should be similar
    compare(models[16], models[13], 1) # should be similar
    compare(models[13], models[12], 1) # should be similar
    
# test(compare_with_seed)

In [ ]:
DEVICE = 'cuda'

def gen_adv_inputs(model, inputs):
    from advertorch.attacks import LinfPGDAttack
    def myloss(yhat, y):
        return -((yhat[:,0]-y[:,0])**2 + 0.1*((yhat[:,1:]-y[:,1:])**2).mean(1)).mean()
        
    model = model.to(DEVICE)
    inputs = torch.from_numpy(inputs).to(DEVICE)
    with torch.no_grad():
        model.eval()
        clean_outputs = model(inputs)
    
    output_shape = clean_outputs.shape
    batch_size = output_shape[0]
    num_classes = output_shape[1]
    
    y = torch.zeros(size=output_shape).to(DEVICE)
    y[:, 0] = 1000
    # more diversity
#     rand_idx = torch.randint(low=0, high=num_classes, size=(batch_size,))
#     y = torch.nn.functional.one_hot(rand_idx, num_classes=num_classes).to(DEVICE) * 10
#     print(y)
    
    adversary = LinfPGDAttack(
        model, loss_fn=myloss, eps=0.1,
        nb_iter=40, eps_iter=0.01, 
        rand_init=True, clip_min=-2.2, clip_max=2.2,
        targeted=False
    )
    
    adv_inputs = adversary.perturb(inputs, y)
    
    with torch.no_grad():
        model.eval()
        adv_outputs = model(adv_inputs).to('cpu').numpy()
#     print(adv_outputs)
    torch.cuda.empty_cache()
    return adv_inputs.to('cpu').numpy()


model = models_dict['pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.1)-prune(0.2)-']
seed_inputs = model.get_seed_inputs(100, rand=False)
seed_outputs = model.batch_forward(seed_inputs)
_, seed_preds = seed_outputs.to('cpu').data.max(1)

adv_inputs = gen_adv_inputs(model.torch_model, seed_inputs)
adv_outputs = model.batch_forward(adv_inputs)
_, adv_preds = adv_outputs.to('cpu').data.max(1)

print(f"seed_preds={seed_preds}, adv_preds={adv_preds}")

In [67]:
DEVICE = 'cuda'
image_size = 224

def expand_vector(x, size):
    batch_size = x.size(0)
    x = x.view(-1, 3, size, size)
    z = torch.zeros(batch_size, 3, image_size, image_size).to(DEVICE)
    z[:, :, :size, :size] = x
    return z

def normalize(x):
    return utils.apply_normalization(x, 'imagenet')

def get_probs(model, x, y):
    output = model(x)
    probs = torch.index_select(torch.nn.functional.softmax(output, -1).data, 1, y)
    return torch.diag(probs).cpu()

def get_preds(model, x):
    output = model(x)
    _, preds = output.data.max(1)
    return preds

# runs simba on a batch of images <images_batch> with true labels (for untargeted attack) or target labels
# (for targeted attack) <labels_batch>
def dct_attack_batch(model, images_batch, labels_batch, max_iters, freq_dims, stride, epsilon, order='rand', targeted=False, pixel_attack=False, log_every=1):
    batch_size = images_batch.size(0)
    image_size = images_batch.size(2)
    # sample a random ordering for coordinates independently per batch element
    if order == 'rand':
        indices = torch.randperm(3 * freq_dims * freq_dims)[:max_iters]
    elif order == 'diag':
        indices = utils.diagonal_order(image_size, 3)[:max_iters]
    elif order == 'strided':
        indices = utils.block_order(image_size, 3, initial_size=freq_dims, stride=stride)[:max_iters]
    else:
        indices = utils.block_order(image_size, 3)[:max_iters]
    if order == 'rand':
        expand_dims = freq_dims
    else:
        expand_dims = image_size
    n_dims = 3 * expand_dims * expand_dims
    x = torch.zeros(batch_size, n_dims)
    # logging tensors
    probs = torch.zeros(batch_size, max_iters)
    succs = torch.zeros(batch_size, max_iters)
    queries = torch.zeros(batch_size, max_iters)
    l2_norms = torch.zeros(batch_size, max_iters)
    linf_norms = torch.zeros(batch_size, max_iters)
    prev_probs = get_probs(model, images_batch, labels_batch)
    preds = get_preds(model, images_batch)
    if pixel_attack:
        trans = lambda z: z
    else:
        trans = lambda z: utils.block_idct(z, block_size=image_size, linf_bound=args.linf_bound)
    remaining_indices = torch.arange(0, batch_size).long()
    for k in range(max_iters):
        dim = indices[k]
        expanded = (images_batch[remaining_indices] + trans(expand_vector(x[remaining_indices], expand_dims))).clamp(-2.6, 2.6)
        perturbation = trans(expand_vector(x, expand_dims))
        l2_norms[:, k] = perturbation.view(batch_size, -1).norm(2, 1)
        linf_norms[:, k] = perturbation.view(batch_size, -1).abs().max(1)[0]
        preds_next = get_preds(model, expanded)
        preds[remaining_indices] = preds_next
        if targeted:
            remaining = preds.ne(labels_batch)
        else:
            remaining = preds.eq(labels_batch)
        # check if all images are misclassified and stop early
        if remaining.sum() == 0:
            adv = (images_batch + trans(expand_vector(x, expand_dims))).clamp(0, 1)
            probs_k = get_probs(model, adv, labels_batch)
            probs[:, k:] = probs_k.unsqueeze(1).repeat(1, max_iters - k)
            succs[:, k:] = torch.ones(batch_size, max_iters - k)
            queries[:, k:] = torch.zeros(batch_size, max_iters - k)
            break
        remaining_indices = torch.arange(0, batch_size)[remaining].long()
        if k > 0:
            succs[:, k-1] = ~remaining
        diff = torch.zeros(remaining.sum(), n_dims)
        diff[:, dim] = epsilon
        left_vec = x[remaining_indices] - diff
        right_vec = x[remaining_indices] + diff
        # trying negative direction
        adv = (images_batch[remaining_indices] + trans(expand_vector(left_vec, expand_dims))).clamp(-2.6, 2.6)
        left_probs = get_probs(model, adv, labels_batch[remaining_indices])
        queries_k = torch.zeros(batch_size)
        # increase query count for all images
        queries_k[remaining_indices] += 1
        if targeted:
            improved = left_probs.gt(prev_probs[remaining_indices])
        else:
            improved = left_probs.lt(prev_probs[remaining_indices])
        # only increase query count further by 1 for images that did not improve in adversarial loss
        if improved.sum() < remaining_indices.size(0):
            queries_k[remaining_indices[~improved]] += 1
        # try positive directions
        adv = (images_batch[remaining_indices] + trans(expand_vector(right_vec, expand_dims))).clamp(-2.6, 2.6)
        right_probs = get_probs(model, adv, labels_batch[remaining_indices])
        if targeted:
            right_improved = right_probs.gt(torch.max(prev_probs[remaining_indices], left_probs))
        else:
            right_improved = right_probs.lt(torch.min(prev_probs[remaining_indices], left_probs))
        probs_k = prev_probs.clone()
        # update x depending on which direction improved
        if improved.sum() > 0:
            left_indices = remaining_indices[improved]
            left_mask_remaining = improved.unsqueeze(1).repeat(1, n_dims)
            x[left_indices] = left_vec[left_mask_remaining].view(-1, n_dims)
            probs_k[left_indices] = left_probs[improved]
        if right_improved.sum() > 0:
            right_indices = remaining_indices[right_improved]
            right_mask_remaining = right_improved.unsqueeze(1).repeat(1, n_dims)
            x[right_indices] = right_vec[right_mask_remaining].view(-1, n_dims)
            probs_k[right_indices] = right_probs[right_improved]
        probs[:, k] = probs_k
        queries[:, k] = queries_k
        prev_probs = probs[:, k]
        if (k + 1) % log_every == 0 or k == max_iters - 1:
            print('Iteration %d: queries = %.4f, prob = %.4f, remaining = %.4f' % (
                    k + 1, queries.sum(1).mean(), probs[:, k].mean(), remaining.float().mean()))
    expanded = (images_batch + trans(expand_vector(x, expand_dims))).clamp(-2.6, 2.6)
    preds = get_preds(model, expanded)
    if targeted:
        remaining = preds.ne(labels_batch)
    else:
        remaining = preds.eq(labels_batch)
    succs[:, max_iters-1] = ~remaining
    return expanded, probs, succs, queries, l2_norms, linf_norms


def search_adv_inputs(model, inputs, labels):
    images_batch = torch.from_numpy(inputs).to(DEVICE)
#     labels_batch = torch.zeros(len(inputs)).long().to(DEVICE)
    labels_batch = labels.long().to(DEVICE)
    with torch.no_grad():
        adv, probs, succs, queries, l2_norms, linf_norms = dct_attack_batch(
            model, images_batch=images_batch, labels_batch=labels_batch,
            max_iters=5000, freq_dims=image_size, stride=7, epsilon=0.2, order='rand',
            targeted=False, pixel_attack=True, log_every=10
        )
    return adv


model = models_dict['pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.1)-prune(0.2)-']
seed_inputs = model.get_seed_inputs(100, rand=False)
seed_outputs = model.batch_forward(seed_inputs)
_, seed_preds = seed_outputs.to('cpu').data.max(1)

adv_inputs = search_adv_inputs(model.torch_model, seed_inputs, seed_preds)
adv_outputs = model.batch_forward(adv_inputs)
_, adv_preds = adv_outputs.to('cpu').data.max(1)

print(f"seed_preds={seed_preds}, adv_preds={adv_preds}")


Iteration 10: queries = 15.3700, prob = 0.9794, remaining = 1.0000
Iteration 20: queries = 30.5300, prob = 0.9790, remaining = 1.0000
Iteration 30: queries = 45.7600, prob = 0.9787, remaining = 1.0000
Iteration 40: queries = 61.2900, prob = 0.9785, remaining = 1.0000
Iteration 50: queries = 76.3900, prob = 0.9782, remaining = 1.0000
Iteration 60: queries = 91.9500, prob = 0.9779, remaining = 1.0000
Iteration 70: queries = 107.3400, prob = 0.9774, remaining = 1.0000


KeyboardInterrupt: 

In [119]:
import torch
import torchvision
import torch.nn.functional as F
import numpy as np
import logging
import copy
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(name)-12s %(levelname)-8s %(message)s")

from benchmark import ImageBenchmark
bench = ImageBenchmark()
models = list(bench.list_models())
models_dict = {}
for i, model in enumerate(models):
    if not model.torch_model_exists():
        continue
#     print(f'{i}\t {model.__str__()}')
    models_dict[model.__str__()] = model


DEVICE = 'cuda'
image_size = 224

def expand_vector(x, image_size):
    batch_size = x.size(0)
    x = x.view(-1, 3, image_size, image_size)
    z = torch.zeros(batch_size, 3, image_size, image_size).to(DEVICE)
    z[:, :, :image_size, :image_size] = x
    return z

def evaluate_inputs(model, inputs, seed_outputs, seed_preds, lambda1=0.5):
    outputs = model(inputs)
    _, preds = outputs.data.max(1)
    changed = preds.ne(seed_preds)
    outputs = torch.nn.functional.softmax(outputs, -1)
    seed_outputs = torch.nn.functional.softmax(seed_outputs, -1)
    reduce_dims = tuple(range(outputs.dim())[1:])
    divergence_arr = torch.sum((outputs - seed_outputs) ** 2, dim=reduce_dims) ** 0.5
    divergence = torch.mean(divergence_arr)
#     divergence_cos = F.cosine_similarity(outputs, seed_outputs)
#     divergence_kld = F.kl_div(seed_outputs, outputs, reduction='none')
#     print(divergence_kld)
    diversity_matrix = torch.cdist(outputs, outputs, p=2.0)
    diversity = torch.mean(diversity_matrix)
    quantile = lambda t, q: t.view(-1).kthvalue(1 + round(float(q) * (t.numel() - 1))).values.item()
    diversity_quantile = quantile(diversity_matrix, 0.011)
#     diversity = diversity ** 2
    score = divergence + lambda1 * diversity
    succ = preds.ne(seed_preds)
    low_divergence_indices = list(torch.nonzero(divergence_arr.lt(divergence)).cpu().numpy())
    low_diversity_indices = list(torch.nonzero(diversity_matrix.lt(diversity_quantile)).cpu().numpy())
    remaining_indices = set()
#     print(changed)
    for i in low_divergence_indices:
        if not changed[i].cpu():
            remaining_indices.add(i[0])
    for i in low_diversity_indices:
        if i[0] == i[1]:
            continue
        if not changed[i[0]]:
            remaining_indices.add(i[0])
        if not changed[i[1]]:
            remaining_indices.add(i[1])
    remaining_indices = sorted(remaining_indices)
#     print(f' low_divergence_indices={len(low_divergence_indices)}\n low_diversity_indices={len(low_diversity_indices)}\n remaining_indices={len(remaining_indices)}')
    eval_line = f'score={score:.4f}, divergence={divergence:.4f}, diversity={diversity:.4f}, num_succ={succ.sum()}, num_remain={len(remaining_indices)}'
    return {
        'outputs': outputs,
        'preds': preds,
        'score': score,
        'divergence': divergence,
        'diversity': diversity,
        'succ': succ,
        'remaining': remaining_indices,
        'eval_line': eval_line
    }
    
    
def optimize_towards_goal(model, seed_inputs, seed_outputs, seed_preds, max_iters=10000, epsilon=0.5, lambda1=0.0, log_every=100, save_every=1000):
#     seed_inputs = torch.from_numpy(seed_inputs).to(DEVICE)
#     seed_outputs = torch.from_numpy(seed_outputs).to(DEVICE)
#     seed_preds = torch.from_numpy(seed_preds).to(DEVICE)
    input_shape = seed_inputs[0].shape
    n_inputs = seed_inputs.shape[0]
    ndims = np.prod(input_shape)

    with torch.no_grad():
        inputs = copy.copy(seed_inputs)
        saved_inputs = {}
        evaluation = evaluate_inputs(model, inputs, seed_outputs, seed_preds, lambda1)
        print(f'initial_evaluation: {evaluation["eval_line"]}')

        for i in range(max_iters):
#             print(f'mutation {i}-th iteration')

            # mutation_pos = np.random.randint(0, ndims)
            mutation_pos = np.random.choice(ndims, size=100, replace=False)
            mutation = np.zeros(ndims).astype(np.float32)
            mutation[mutation_pos] = epsilon
            mutation = np.reshape(mutation, input_shape)

            mutation_batch = np.zeros(shape=inputs.shape).astype(np.float32)
#             all_indices = list(range(0, n_inputs))
#             mutation_indices = np.random.choice(all_indices, size=int(n_inputs * 0.85), replace=False)
#             print(mutation_indices)
#             mutation_idx = np.random.randint(0, n_inputs)
            mutation_indices = evaluation['remaining']
            if len(mutation_indices) == 0:
                print(f'{i:4d}-th - no remaining indice: {evaluation["eval_line"]}')
                break
#             print(mutation_indices)
            mutation_batch[mutation_indices] = mutation
            mutation_batch = torch.from_numpy(mutation_batch).to(DEVICE)

            prev_score = evaluation["score"]
            mutate_right_inputs = (inputs + mutation_batch).clamp(-2.6, 2.6)
            mutate_right_eval = evaluate_inputs(model, mutate_right_inputs, seed_outputs, seed_preds)
            mutate_right_score = mutate_right_eval['score']
            mutate_left_inputs = (inputs - mutation_batch).clamp(-2.6, 2.6)
            mutate_left_eval = evaluate_inputs(model, mutate_left_inputs, seed_outputs, seed_preds)
            mutate_left_score = mutate_left_eval['score']

            if mutate_right_score <= prev_score and mutate_left_score <= prev_score:
                pass
            elif mutate_right_score > mutate_left_score:
#                 print(f'mutate right: {prev_score}->{mutate_right_score}')
                inputs = mutate_right_inputs
                evaluation = mutate_right_eval
            else:
#                 print(f'mutate left: {prev_score}->{mutate_left_score}')
                inputs = mutate_left_inputs
                evaluation = mutate_left_eval
            if i % log_every == 0:
                print(f'{i:4d}-th evaluation: {evaluation["eval_line"]}')
            if i % save_every == 0:
                saved_inputs[i] = copy.copy(inputs.cpu().numpy())
        return inputs, saved_inputs

model = models_dict['pretrain(resnet18,ImageNet)-transfer(Flower102,0.1)-prune(0.2)-']
seed_inputs = model.get_seed_inputs(100, rand=False)
seed_inputs = torch.from_numpy(seed_inputs).to(DEVICE)
seed_outputs = model.batch_forward(seed_inputs)
_, seed_preds = seed_outputs.data.max(1)

adv_inputs, _ = optimize_towards_goal(model.torch_model_on_device, seed_inputs, seed_outputs, seed_preds, log_every=100)
adv_outputs = model.batch_forward(adv_inputs).cpu()
_, adv_preds = adv_outputs.data.max(1)

print(f"seed_preds={seed_preds}, adv_preds={adv_preds}")

initial_evaluation: score=0.0000, divergence=0.0000, diversity=1.3336, num_succ=0, num_remain=10
   0-th evaluation: score=0.6671, divergence=0.0002, diversity=1.3337, num_succ=0, num_remain=87
 100-th evaluation: score=0.6848, divergence=0.0379, diversity=1.2937, num_succ=1, num_remain=39
 200-th evaluation: score=0.6887, divergence=0.0508, diversity=1.2758, num_succ=1, num_remain=30
 300-th evaluation: score=0.6948, divergence=0.0707, diversity=1.2482, num_succ=1, num_remain=38
 400-th evaluation: score=0.7072, divergence=0.1054, diversity=1.2037, num_succ=1, num_remain=33
 500-th evaluation: score=0.7203, divergence=0.1417, diversity=1.1571, num_succ=1, num_remain=23
 600-th evaluation: score=0.7380, divergence=0.1865, diversity=1.1029, num_succ=1, num_remain=34
 700-th evaluation: score=0.7486, divergence=0.2157, diversity=1.0658, num_succ=1, num_remain=30
 800-th evaluation: score=0.7665, divergence=0.2593, diversity=1.0145, num_succ=1, num_remain=23
 900-th evaluation: score=0.78

KeyboardInterrupt: 

In [ ]:
from scipy import spatial
from utils import lazy_property, Utils
import os


def compute_ddv(model, normal_outputs, adv_outputs):
    output_pairs = zip(normal_outputs, adv_outputs)
    # print(list(output_pairs)[0])
    ddv = []  # DDV is short for decision distance vector
    for i, (ya, yb) in enumerate(output_pairs):
        dist = spatial.distance.cosine(ya, yb)
        ddv.append(dist)
    ddv = Utils.normalize(np.array(ddv))
    return ddv


for i, model in enumerate(models):
    if not model.torch_model_exists():
        continue
    model_name = model.__str__()
    model_path = model.torch_model_path
    if i < 6: # skip pretrained models
        continue
    if 'quantize' in model_name: # skip quantized models
        continue
    print(f'{i}\t generating inputs for {model.__str__()}')
    seed_inputs = model.get_seed_inputs(100, rand=False)
    seed_inputs = torch.from_numpy(seed_inputs).to(DEVICE)
    seed_outputs = model.batch_forward(seed_inputs)
    _, seed_preds = seed_outputs.data.max(1)

    adv_inputs, saved_inputs = optimize_towards_goal(
        model.torch_model_on_device, seed_inputs, seed_outputs, seed_preds,
        epsilon=0.2, lambda1=0.5, max_iters=10000, log_every=100, save_every=1000)
    adv_outputs = model.batch_forward(adv_inputs).cpu()
    _, adv_preds = adv_outputs.data.max(1)
    
    ddv = compute_ddv(model, seed_outputs.cpu().numpy(), adv_outputs.cpu().numpy())
    print(ddv)
    
    out_path = os.path.join(model_path, 'inputs.npz')
    np.savez(out_path, seed_inputs=seed_inputs.cpu().numpy(), adv_inputs=adv_inputs.cpu().numpy(), ddv=ddv, saved_inputs=saved_inputs)
    # break

6	 generating inputs for pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.1)-
initial_evaluation: score=0.6356, divergence=0.0000, diversity=1.2712, num_succ=0, num_remain=10
   0-th evaluation: score=0.6359, divergence=0.0003, diversity=1.2713, num_succ=0, num_remain=89
 100-th evaluation: score=0.6460, divergence=0.0121, diversity=1.2679, num_succ=0, num_remain=34
 200-th evaluation: score=0.6478, divergence=0.0144, diversity=1.2668, num_succ=0, num_remain=35
 300-th evaluation: score=0.6481, divergence=0.0148, diversity=1.2667, num_succ=0, num_remain=33
 400-th evaluation: score=0.6488, divergence=0.0156, diversity=1.2664, num_succ=0, num_remain=33
 500-th evaluation: score=0.6490, divergence=0.0159, diversity=1.2661, num_succ=0, num_remain=33
 600-th evaluation: score=0.6492, divergence=0.0164, diversity=1.2657, num_succ=0, num_remain=33
 700-th evaluation: score=0.6495, divergence=0.0166, diversity=1.2657, num_succ=0, num_remain=31
 800-th evaluation: score=0.6498, divergence=0.017

8300-th evaluation: score=0.6810, divergence=0.0709, diversity=1.2202, num_succ=0, num_remain=24
8400-th evaluation: score=0.6811, divergence=0.0710, diversity=1.2202, num_succ=0, num_remain=23
8500-th evaluation: score=0.6812, divergence=0.0712, diversity=1.2199, num_succ=0, num_remain=23
8600-th evaluation: score=0.6813, divergence=0.0713, diversity=1.2199, num_succ=0, num_remain=23
8700-th evaluation: score=0.6814, divergence=0.0714, diversity=1.2200, num_succ=0, num_remain=24
8800-th evaluation: score=0.6814, divergence=0.0715, diversity=1.2198, num_succ=0, num_remain=23
8900-th evaluation: score=0.6816, divergence=0.0719, diversity=1.2194, num_succ=0, num_remain=24
9000-th evaluation: score=0.6817, divergence=0.0720, diversity=1.2193, num_succ=0, num_remain=23
9100-th evaluation: score=0.6817, divergence=0.0720, diversity=1.2194, num_succ=0, num_remain=23
9200-th evaluation: score=0.6818, divergence=0.0721, diversity=1.2194, num_succ=0, num_remain=23
9300-th evaluation: score=0.68

5500-th evaluation: score=0.7079, divergence=0.0862, diversity=1.2435, num_succ=2, num_remain=18
5600-th evaluation: score=0.7083, divergence=0.0873, diversity=1.2421, num_succ=2, num_remain=17
5700-th evaluation: score=0.7085, divergence=0.0876, diversity=1.2418, num_succ=2, num_remain=16
5800-th evaluation: score=0.7089, divergence=0.0891, diversity=1.2395, num_succ=2, num_remain=19
5900-th evaluation: score=0.7091, divergence=0.0897, diversity=1.2388, num_succ=2, num_remain=17
6000-th evaluation: score=0.7099, divergence=0.0922, diversity=1.2354, num_succ=2, num_remain=21
6100-th evaluation: score=0.7103, divergence=0.0932, diversity=1.2342, num_succ=2, num_remain=17
6200-th evaluation: score=0.7107, divergence=0.0943, diversity=1.2329, num_succ=2, num_remain=18
6300-th evaluation: score=0.7121, divergence=0.0986, diversity=1.2270, num_succ=2, num_remain=17
6400-th evaluation: score=0.7129, divergence=0.1013, diversity=1.2233, num_succ=2, num_remain=18
6500-th evaluation: score=0.71

2700-th evaluation: score=0.6971, divergence=0.0353, diversity=1.3237, num_succ=0, num_remain=16
2800-th evaluation: score=0.6972, divergence=0.0353, diversity=1.3237, num_succ=0, num_remain=16
2900-th evaluation: score=0.6973, divergence=0.0355, diversity=1.3236, num_succ=0, num_remain=18
3000-th evaluation: score=0.6973, divergence=0.0357, diversity=1.3234, num_succ=0, num_remain=18
3100-th evaluation: score=0.6974, divergence=0.0357, diversity=1.3233, num_succ=0, num_remain=19
3200-th evaluation: score=0.6974, divergence=0.0358, diversity=1.3231, num_succ=0, num_remain=20
3300-th evaluation: score=0.6975, divergence=0.0361, diversity=1.3227, num_succ=0, num_remain=22
3400-th evaluation: score=0.6976, divergence=0.0366, diversity=1.3220, num_succ=0, num_remain=24
3500-th evaluation: score=0.6977, divergence=0.0369, diversity=1.3217, num_succ=0, num_remain=20
3600-th evaluation: score=0.6979, divergence=0.0372, diversity=1.3214, num_succ=0, num_remain=17
3700-th evaluation: score=0.69

9	 generating inputs for pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.1)-
initial_evaluation: score=0.4123, divergence=0.0000, diversity=0.8246, num_succ=0, num_remain=8
   0-th evaluation: score=0.4128, divergence=0.0005, diversity=0.8247, num_succ=0, num_remain=87
 100-th evaluation: score=0.4452, divergence=0.0325, diversity=0.8254, num_succ=10, num_remain=22
 200-th evaluation: score=0.4538, divergence=0.0405, diversity=0.8266, num_succ=13, num_remain=13
 300-th evaluation: score=0.4589, divergence=0.0448, diversity=0.8282, num_succ=13, num_remain=8
 400-th evaluation: score=0.4601, divergence=0.0461, diversity=0.8281, num_succ=15, num_remain=4
 500-th evaluation: score=0.4602, divergence=0.0462, diversity=0.8282, num_succ=15, num_remain=5
 600-th evaluation: score=0.4604, divergence=0.0464, diversity=0.8280, num_succ=15, num_remain=4
 700-th evaluation: score=0.4620, divergence=0.0483, diversity=0.8275, num_succ=15, num_remain=5
 800-th evaluation: score=0.4622, divergence=0.0485

...

2700-th evaluation: score=0.9361, divergence=0.5560, diversity=0.7601, num_succ=28, num_remain=5
2800-th evaluation: score=0.9381, divergence=0.5592, diversity=0.7577, num_succ=29, num_remain=8
2900-th evaluation: score=0.9403, divergence=0.5627, diversity=0.7551, num_succ=30, num_remain=4
3000-th evaluation: score=0.9417, divergence=0.5647, diversity=0.7541, num_succ=30, num_remain=5
3100-th evaluation: score=0.9426, divergence=0.5656, diversity=0.7539, num_succ=30, num_remain=4
3200-th evaluation: score=0.9436, divergence=0.5673, diversity=0.7526, num_succ=30, num_remain=4
3300-th evaluation: score=0.9442, divergence=0.5683, diversity=0.7519, num_succ=30, num_remain=3
3400-th evaluation: score=0.9448, divergence=0.5688, diversity=0.7520, num_succ=30, num_remain=3
3500-th evaluation: score=0.9450, divergence=0.5690, diversity=0.7520, num_succ=30, num_remain=3
3600-th evaluation: score=0.9452, divergence=0.5691, diversity=0.7522, num_succ=30, num_remain=3
3700-th evaluation: score=0.94

51	 generating inputs for pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.1)-prune(0.2)-
initial_evaluation: score=0.5554, divergence=0.0000, diversity=1.1108, num_succ=0, num_remain=9
   0-th evaluation: score=0.5561, divergence=0.0006, diversity=1.1110, num_succ=0, num_remain=88
 100-th evaluation: score=0.5798, divergence=0.0268, diversity=1.1059, num_succ=1, num_remain=22
 200-th evaluation: score=0.5876, divergence=0.0366, diversity=1.1019, num_succ=3, num_remain=20
 300-th evaluation: score=0.5933, divergence=0.0438, diversity=1.0990, num_succ=5, num_remain=15
 400-th evaluation: score=0.5941, divergence=0.0451, diversity=1.0981, num_succ=5, num_remain=12
 500-th evaluation: score=0.5966, divergence=0.0480, diversity=1.0972, num_succ=5, num_remain=13
 600-th evaluation: score=0.6003, divergence=0.0528, diversity=1.0951, num_succ=5, num_remain=17
 700-th evaluation: score=0.6057, divergence=0.0592, diversity=1.0930, num_succ=6, num_remain=14
 800-th evaluation: score=0.6101, diverge

8300-th evaluation: score=0.6870, divergence=0.1563, diversity=1.0614, num_succ=10, num_remain=12
8400-th evaluation: score=0.6873, divergence=0.1566, diversity=1.0613, num_succ=10, num_remain=12
8500-th evaluation: score=0.6873, divergence=0.1567, diversity=1.0613, num_succ=10, num_remain=12
8600-th evaluation: score=0.6873, divergence=0.1567, diversity=1.0613, num_succ=10, num_remain=12
8700-th evaluation: score=0.6873, divergence=0.1567, diversity=1.0613, num_succ=10, num_remain=12
8800-th evaluation: score=0.6873, divergence=0.1567, diversity=1.0613, num_succ=10, num_remain=12
8900-th evaluation: score=0.6874, divergence=0.1567, diversity=1.0613, num_succ=10, num_remain=12
9000-th evaluation: score=0.6874, divergence=0.1567, diversity=1.0613, num_succ=10, num_remain=12
9100-th evaluation: score=0.6874, divergence=0.1567, diversity=1.0613, num_succ=10, num_remain=12
9200-th evaluation: score=0.6874, divergence=0.1568, diversity=1.0614, num_succ=10, num_remain=12
9300-th evaluation: 

5000-th evaluation: score=0.6920, divergence=0.1842, diversity=1.0156, num_succ=9, num_remain=13
5100-th evaluation: score=0.6922, divergence=0.1843, diversity=1.0157, num_succ=9, num_remain=13
5200-th evaluation: score=0.6923, divergence=0.1844, diversity=1.0158, num_succ=9, num_remain=13
5300-th evaluation: score=0.6925, divergence=0.1845, diversity=1.0159, num_succ=9, num_remain=13
5400-th evaluation: score=0.6926, divergence=0.1846, diversity=1.0159, num_succ=9, num_remain=13
5500-th evaluation: score=0.6927, divergence=0.1847, diversity=1.0160, num_succ=9, num_remain=13
5600-th evaluation: score=0.6927, divergence=0.1847, diversity=1.0161, num_succ=9, num_remain=13
5700-th evaluation: score=0.6928, divergence=0.1847, diversity=1.0161, num_succ=9, num_remain=13
5800-th evaluation: score=0.6929, divergence=0.1848, diversity=1.0161, num_succ=9, num_remain=13
5900-th evaluation: score=0.6929, divergence=0.1849, diversity=1.0161, num_succ=9, num_remain=13
6000-th evaluation: score=0.69

1700-th evaluation: score=0.4247, divergence=0.0705, diversity=0.7083, num_succ=19, num_remain=2
1800-th evaluation: score=0.4247, divergence=0.0705, diversity=0.7084, num_succ=19, num_remain=2
1900-th evaluation: score=0.4248, divergence=0.0706, diversity=0.7084, num_succ=19, num_remain=2
2000-th evaluation: score=0.4248, divergence=0.0706, diversity=0.7084, num_succ=19, num_remain=2
2100-th evaluation: score=0.4248, divergence=0.0706, diversity=0.7084, num_succ=19, num_remain=2
2200-th evaluation: score=0.4248, divergence=0.0706, diversity=0.7085, num_succ=19, num_remain=2
2300-th evaluation: score=0.4249, divergence=0.0706, diversity=0.7084, num_succ=19, num_remain=2
2400-th evaluation: score=0.4249, divergence=0.0706, diversity=0.7085, num_succ=19, num_remain=2
2500-th evaluation: score=0.4249, divergence=0.0707, diversity=0.7085, num_succ=19, num_remain=2
2600-th evaluation: score=0.4249, divergence=0.0707, diversity=0.7085, num_succ=19, num_remain=2
2700-th evaluation: score=0.42

In [88]:
DEVICE = 'cuda'
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(name)-12s %(levelname)-8s %(message)s")


def gen_adv_inputs(model, inputs):
    from advertorch.attacks import LinfPGDAttack
    def myloss(yhat, y):
        return -((yhat[:,0]-y[:,0])**2 + 0.1*((yhat[:,1:]-y[:,1:])**2).mean(1)).mean()
        
    model = model.to(DEVICE)
    inputs = torch.from_numpy(inputs).to(DEVICE)
    with torch.no_grad():
        model.eval()
        clean_outputs = model(inputs)
    
    output_shape = clean_outputs.shape
    batch_size = output_shape[0]
    num_classes = output_shape[1]
    
    y = torch.zeros(size=output_shape).to(DEVICE)
    y[:, 0] = 1000
    # more diversity
#     rand_idx = torch.randint(low=0, high=num_classes, size=(batch_size,))
#     y = torch.nn.functional.one_hot(rand_idx, num_classes=num_classes).to(DEVICE) * 10
#     print(y)
    
    adversary = LinfPGDAttack(
        model, loss_fn=myloss, eps=0.1,
        nb_iter=40, eps_iter=0.01, 
        rand_init=True, clip_min=-2.2, clip_max=2.2,
        targeted=False
    )
    
    adv_inputs = adversary.perturb(inputs, y)
    
    with torch.no_grad():
        model.eval()
        adv_outputs = model(adv_inputs).to('cpu').numpy()
#     print(adv_outputs)
    torch.cuda.empty_cache()
    return adv_inputs.to('cpu').numpy()


def compare_with_adv(model1, model2, truth=-1):
    if truth == -1:
        truth = 1 if model1.__str__().split('-')[0] == model2.__str__().split('-')[0] else 0
    print(f'comparing:\n  model1={model1}\n  model2={model2}')
    md = ModelDiff(model1, model2)
    rand = False
    seed_inputs1 = model1.get_seed_inputs(100, rand=rand)
    seed_inputs2 = model2.get_seed_inputs(100, rand=rand)
    seed_inputs = np.concatenate([seed_inputs1, seed_inputs2])
    
    adv_inputs1 = gen_adv_inputs(model1.torch_model, seed_inputs1)
    adv_inputs2 = gen_adv_inputs(model2.torch_model, seed_inputs2)
    adv_inputs = np.concatenate([adv_inputs1, adv_inputs2])

    hybrid_inputs = np.concatenate([seed_inputs1, adv_inputs1])
    
    sim = md.compute_similarity_with_ddv(adv_inputs)
    print(f' compute_similarity_with_ddv,adv_inputs is {sim}, truth is {truth}')
    sim = md.compute_similarity_with_ddv(seed_inputs)
    print(f' compute_similarity_with_ddv,seed_inputs is {sim}, truth is {truth}')
    sim = md.compute_similarity_with_ddv(hybrid_inputs)
    print(f' compute_similarity_with_ddv,hybrid_inputs is {sim}, truth is {truth}')

    
def compare_with_inputs(model1, model2, truth=-1):
    if truth == -1:
        truth = 1 if model1.__str__().split('-')[0] == model2.__str__().split('-')[0] else 0
    print(f'comparing:\n  model1={model1}\n  model2={model2}')
    md = ModelDiff(model1, model2)
    hybrid_inputs = np.concatenate([seed_inputs.cpu().numpy(), adv_inputs.cpu().numpy()])
    
    model1.torch_model.to(DEVICE)
    model2.torch_model.to(DEVICE)
    
#     sim = md.compute_similarity_with_ddv(adv_inputs)
#     print(f' compare_with_inputs,adv_inputs is {sim}, truth is {truth}')
#     sim = md.compute_similarity_with_ddv(seed_inputs)
#     print(f' compare_with_inputs,seed_inputs is {sim}, truth is {truth}')
    sim = md.compute_similarity_with_ddv(hybrid_inputs)
    print(f' compare_with_inputs,hybrid_inputs is {sim}, truth is {truth}')

# compare_with_adv(models[1], models[17], 1)
test(compare_with_inputs)

2021-01-27 02:48:49,279 ModelDiff    INFO     comparing pretrain(resnet18,ImageNet)- and pretrain(mbnetv2,ImageNet)-
2021-01-27 02:48:49,345 ModelDiff    INFO     computing DDVs


comparing:
  model1=pretrain(resnet18,ImageNet)-
  model2=pretrain(mbnetv2,ImageNet)-


2021-01-27 02:48:49,584 ModelDiff    INFO       DDV computed: shape=(100,) and (100,)
2021-01-27 02:48:49,585 ModelDiff    INFO     measuring model similarity
2021-01-27 02:48:49,588 ModelDiff    INFO       model similarity: 0.9655840570025039
2021-01-27 02:48:49,589 ModelDiff    INFO     comparing pretrain(resnet18,ImageNet)- and pretrain(resnet18,ImageNet)-transfer(SDog120,1)-


 compare_with_inputs,hybrid_inputs is 0.9655840570025039, truth is 0
comparing:
  model1=pretrain(resnet18,ImageNet)-
  model2=pretrain(resnet18,ImageNet)-transfer(SDog120,1)-


2021-01-27 02:48:50,133 ModelDiff    INFO     computing DDVs
2021-01-27 02:48:50,348 ModelDiff    INFO       DDV computed: shape=(100,) and (100,)
2021-01-27 02:48:50,349 ModelDiff    INFO     measuring model similarity
2021-01-27 02:48:50,351 ModelDiff    INFO       model similarity: 0.9007110683750055
2021-01-27 02:48:50,355 ModelDiff    INFO     comparing pretrain(resnet18,ImageNet)-transfer(SDog120,0.5)- and pretrain(resnet18,ImageNet)-transfer(Flower102,0.5)-


 compare_with_inputs,hybrid_inputs is 0.9007110683750055, truth is 1
comparing:
  model1=pretrain(resnet18,ImageNet)-transfer(SDog120,0.5)-
  model2=pretrain(resnet18,ImageNet)-transfer(Flower102,0.5)-


2021-01-27 02:48:50,965 ModelDiff    INFO     computing DDVs
2021-01-27 02:48:51,183 ModelDiff    INFO       DDV computed: shape=(100,) and (100,)
2021-01-27 02:48:51,183 ModelDiff    INFO     measuring model similarity
2021-01-27 02:48:51,186 ModelDiff    INFO       model similarity: 0.8362770628971133
2021-01-27 02:48:51,187 ModelDiff    INFO     comparing pretrain(resnet18,ImageNet)-transfer(Flower102,0.5)- and pretrain(resnet18,ImageNet)-transfer(Flower102,0.1)-


 compare_with_inputs,hybrid_inputs is 0.8362770628971133, truth is 1
comparing:
  model1=pretrain(resnet18,ImageNet)-transfer(Flower102,0.5)-
  model2=pretrain(resnet18,ImageNet)-transfer(Flower102,0.1)-


2021-01-27 02:48:51,484 ModelDiff    INFO     computing DDVs
2021-01-27 02:48:51,697 ModelDiff    INFO       DDV computed: shape=(100,) and (100,)
2021-01-27 02:48:51,697 ModelDiff    INFO     measuring model similarity
2021-01-27 02:48:51,700 ModelDiff    INFO       model similarity: 0.9517062442803106


 compare_with_inputs,hybrid_inputs is 0.9517062442803106, truth is 1


In [ ]:
from utils import Utils
from matplotlib import pyplot as plt

def show_images(images, labels, title='examples'):
    images = np.transpose(images, (0, 2, 3, 1))
    plt.figure(figsize=(10,10))
    plt.subplots_adjust(hspace=0.2)
    for n in range(25):
        plt.subplot(5,5,n+1)
        img = images[n]
        img = img.squeeze()
        plt.imshow(img)
        plt.title(f'{labels[n]}')
        plt.axis('off')
    _ = plt.suptitle(title)
    plt.show()

# show_images(seed_inputs, list(range(seed_inputs.shape[0])))
# print(seed_inputs[0])
# batch_outputs1 = model1.batch_forward(seed_inputs)
# batch_preds1 = batch_outputs1.to('cpu').data.max(1)
# print(batch_preds1)

In [ ]:
m = models[2]
mm = m.torch_model
test_loader = m.benchmark.get_dataloader(m.dataset_id, split='test')

# with torch.no_grad():
#     mm.eval()
#     total = 0
#     top1 = 0
#     for i, (batch, label) in enumerate(test_loader):
#         batch, label = batch.to(DEVICE), label.to(DEVICE)
#         total += batch.size(0)
#         out = mm(batch)
#         _, pred = out.max(dim=1)
#         top1 += int(pred.eq(label).sum().item())

# acc = float(top1) / total * 100
# print(top1, total, acc)

with torch.no_grad():
    for i, (images, labels) in enumerate(test_loader):
        if i != 10:
            continue
        images, labels = images.cuda(), labels.cuda()
        outputs = mm(images)
        _, preds = outputs.max(dim=1)
        print(outputs[0])
        labels = [f'{label}-{preds[i]}' for i,label in enumerate(list(labels))]
        show_images(images.to('cpu').numpy(), labels)
        break

In [ ]:
from model.fe_mobilenet import mbnetv2_dropout
import os

torch_model = mbnetv2_dropout(
            pretrained=False,
            num_classes=67
        )
ckpt = torch.load(os.path.join('models', 'train(mbnetv2,MIT67)-', 'ckpt.pth'))
torch_model.load_state_dict(ckpt['state_dict'])
mm = torch_model

In [ ]:
import torch
import torchvision
import numpy as np

from benchmark import ImageBenchmark
bench = ImageBenchmark()
models = list(bench.list_models())
for i, model in enumerate(models):
    if not model.torch_model_exists():
        continue
    if 'quantize' in model.__str__():
        continue
    acc = model.accuracy if model.dataset_id != 'ImageNet' else '-'
    print(f'{i}\t {model.__str__()} {acc}')

In [46]:
npzfile  = np.load(os.path.join('models', 'train(mbnetv2,Flower102)-', 'inputs.npz'), allow_pickle=True)
npzfile.files
saved_inputs = npzfile['saved_inputs']

In [109]:
a = torch.zeros(10)
a[2] = 1
b = torch.zeros(10)
b[3] = 1
a.ne(b).to('cuda')[2].item()

True